In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error, log_loss
from sklearn.cross_validation import train_test_split
from sklearn import cluster
from sklearn.decomposition import PCA
from scipy.stats import itemfreq
from mpl_toolkits.mplot3d import Axes3D


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
%matplotlib inline

In [2]:
!pwd

/home/ec2-user/datakind


In [3]:
%%time
df = pd.read_csv('../data/datakind/data.csv', parse_dates=['reg_date', 'transaction_date'], na_values = 'undefined')

CPU times: user 7.64 s, sys: 656 ms, total: 8.29 s
Wall time: 8.28 s


In [4]:
%%time 
df_cluster = df[df.transaction_date < '2014-01-01 00:00:00.000'][['org', 'item']].set_index('org')
df_cluster = pd.get_dummies(df_cluster.item)
df_cluster = df_cluster.groupby(df_cluster.index).sum()

CPU times: user 55.2 s, sys: 31.5 s, total: 1min 26s
Wall time: 1min 26s


In [5]:
%%time
rows = random.sample(df_cluster.index, int(0.1*len(df_cluster.index)))
df_cluster10 = df_cluster.ix[rows]
#df_cluster90 = df_cluster.drop(rows)

CPU times: user 1.28 s, sys: 248 ms, total: 1.53 s
Wall time: 1.53 s


In [ ]:
X_cluster = df_cluster.values
X_cluster10 = df_cluster10.values
print X_cluster.shape
print X_cluster10.shape

(220938, 4186)
(22093, 4186)


In [ ]:
%%time
pca = PCA(n_components = 100)
pca.fit(X_cluster10)

In [ ]:
X_cluster_pca = pca.transform(X_cluster)

In [ ]:
%%time
km = cluster.KMeans(init='k-means++', n_clusters=20, random_state=2, max_iter = 1000)
km.fit(X_cluster_pca)

km_labels = km.labels_

In [ ]:
x = X_cluster_pca[:,0]
y = X_cluster_pca[:,1]
z = X_cluster_pca[:,2]

In [ ]:
plt.scatter(x,y, c=km_labels, cmap=plt.cm.brg, alpha = .3)

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=45)
ax.scatter(x, y, z, c= km_labels,
           cmap=plt.cm.brg, alpha = .2, s = 50)

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=-45)
ax.scatter(x, y, z, c= km_labels,
           cmap=plt.cm.brg, alpha = .2, s = 50)

In [ ]:
df_org_cluster = pd.DataFrame(zip(df_cluster.index,km_labels),columns = ['org','org_cluster']).set_index('org')


In [ ]:
df_model = df[df.transaction_date >= '2014-01-01 00:00:00.000']

In [ ]:
idx_max_tran = df_model.reset_index().groupby('org')[['index','transaction_date']].max()['index'].tolist()

In [ ]:
df_model = df_model.ix[idx_max_tran][['org','org_type', 'org_subtype', 'budget',
                                      'reg_date', 'role', 'country']].set_index('org')

In [ ]:
df_model['years_since_base'] = df_model.reg_date.map(lambda x: (x - np.datetime64('2000-01-01 00:00:00.000'))/datetime.timedelta(days=1))

In [ ]:
df_model = df_model.drop('reg_date',axis=1)

In [ ]:
%%time
df_items = df[df.transaction_date >= '2014-01-01 00:00:00.000']
df_items = df_items[['org','item']].set_index('org')
df_items = pd.get_dummies(df_items.item)
df_items = df_items.groupby(df_items.index).sum()

In [ ]:
%%time
df_items_sum = df_items.sum()
cols = df_items_sum[df_items_sum>=100].index

In [ ]:
df_y = df_items.ix[df_model.index,cols]

In [ ]:
df_X = df_model.join([df_org_cluster])

In [ ]:
print df_X.shape
print df_y.shape


In [ ]:
#assign new customers to group 20 (21st group since 0 index)
df_X.loc[df_X.org_cluster.isnull(), 'org_cluster'] = 20
df_X.loc[df_X.role.isnull(), 'role'] = 0
df_X.loc[(df_X.budget < 0)|(df_X.budget.isnull()), 'budget'] = 0


In [ ]:
for i in ['org_subtype','country']:
    label_encoder = preprocessing.LabelEncoder()
    df_X[i] = label_encoder.fit_transform(df_X[i])

In [ ]:
for i in df_y.columns:
    df_y.loc[df_y[i]>0,i]=1

In [ ]:
X = df_X.values
y = df_y.values

In [ ]:
df_X.info()
print [ 0.04376719,0.11020841,0.2507667 ,0.05899064,0.00188967,0.45510355 ,0.07927384]

In [ ]:
rf_log_loss_scores = {}
rf_y_predicted_probas = {}
rf_estimators = {}
rf_clfs = {}
rf_feature_importances = {}
X_tests = {}
y_tests = {}


In [ ]:
%%time
for i in range len(y[1]):
        y_iter = y[:,i]

        X_train, X_test, y_train, y_test = train_test_split(X, y_iter, test_size = 0.25, random_state=99)
        
        clf = RandomForestClassifier(n_jobs = -1, n_estimators=500)
        clf.fit(X_train, y_train)

        y_predicted_proba = clf.predict_proba(X_test)
        log_loss_score = log_loss(y_test, y_predicted_proba)
        
        rf_y_predicted_probas[i] = y_predicted_proba
        rf_log_loss_scores[i] = log_loss_score
        rf_clfs[i] = clf
        rf_estimators[i] = clf.estimators_
        rf_feature_importances[i] = clf.feature_importances_
        X_tests[i] = X_test
        y_tests[i] = y_test

In [ ]:
import pickle

with open('rf_log_loss_scores.p', 'wb') as handle:
    pickle.dump(rf_log_loss_scores, handle)

with open('rf_y_predicted_probas.p', 'wb') as handle:
    pickle.dump(rf_y_predicted_probas, handle)
    
with open('rf_estimators.p', 'wb') as handle:
    pickle.dump(rf_estimators, handle)

with open('rf_clfs.p', 'wb') as handle:
    pickle.dump(rf_clfs, handle)

with open('rf_feature_importances.p', 'wb') as handle:
    pickle.dump(rf_feature_importances, handle)
    
with open('X_tests.p', 'wb') as handle:
    pickle.dump(X_tests, handle)

with open('y_tests.p', 'wb') as handle:
    pickle.dump(y_tests, handle)
    

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted_proba[:, 1
                                                                         ])
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.figure(figsize = (10,10))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

In [ ]:
plot_roc_curve(y_tests[4], rf_y_predicted_probas[4])